In [15]:
from finance_utils.electrical.china.spot.measure.internal.forecast import ForecastCurve
from finance_utils.electrical.china.spot.measure.trade import ForecastMarket

import numpy
from data_utils.stochastic_utils.distributions.baseclass import ABCDistribution
from data_utils.stochastic_utils.distributions.basic_distributions import NormalDistribution
from data_utils.stochastic_utils.distributions.nonParametricDistribution import HistogramDist, LogHisDist
from data_utils.stochastic_utils.distributions.discrete.basic import DiscreteDistribution
from matplotlib import pyplot
import tqdm
import json

from easy_utils.db_utils.mysql import MySQLLink
from easy_utils.obj_utils.enumerable_utils import flatten

from private.db import mysql_94j6daxc

In [16]:
quantity_min = 0
quantity_max = 30
submitted_min = 0
submitted_max = 30
trigger_rate = 0.05
punishment_rate = 0.5

db = MySQLLink(
    url=mysql_94j6daxc["url"],
    port=mysql_94j6daxc["port"],
    user=mysql_94j6daxc["user"],
    pwd=mysql_94j6daxc["pwd"],
    db="tianrun_etrade"
)
d,_,__ = db.select(
    "select time_order,dayahead_statement_price,realtime_statement_price,dayahead_clearing_quantity,realtime_clearing_quantity "
    "from spot_real "
    "where station_id='14013006' and date between '2024-5-1' and '2024-8-1'"
    "order by date,time_order"
)
historical_array = numpy.array(d)

In [17]:
def get_slice(xlist, order, slice) -> numpy.ndarray:
    return xlist[numpy.where(xlist[:,0]==order)][:,slice]

historcial_dayahead = ForecastCurve(
    [
        HistogramDist(
            get_slice(historical_array, i, 1)
        ) for i in range(1,5)
    ]
)
historical_realtime = ForecastCurve(
    [
        HistogramDist(
            get_slice(historical_array, i, 2)
        ) for i in range(1,5)
    ]
)

historical_quantity = ForecastCurve(
    [
        HistogramDist(
            numpy.sum(get_slice(historical_array, 1, [3,4]), axis=1)
        ) for i in range(1,5)
    ]
)
historical_market = ForecastMarket(historcial_dayahead, historical_realtime, historical_quantity)

In [18]:
s_train = historical_market.self_related_random_sample([0.9] * 4, 100)
numpy.max(flatten(s_train.quantity_sample))

19.114037995936332

In [19]:
print(
    historical_market.self_related_random_sample([0.9] * 4, 100).differential_evolution__search(submitted_min=0,submitted_max=20,trigger_rate=trigger_rate, punishment_rate=punishment_rate)
)

SearchResult(y=array([19.86701772,  0.1492182 ,  6.66549994,  0.52190155]), total=473008.56672625925)


In [26]:
d2,_,__ = db.select(
    "select time_order,dayahead_statement_price,realtime_statement_price,dayahead_clearing_quantity,realtime_clearing_quantity "
    "from spot_real "
    "where station_id='14013006' and date between '2024-8-1' and '2024-9-1'"
    "order by date,time_order"
)
future_array = numpy.array(d2)
future_dayahead = [
    get_slice(future_array, i, 1) for i in range(1,5)
]
future_dayahead_curve = ForecastCurve([HistogramDist(d) for d in future_dayahead])

future_realtime = [
    get_slice(future_array, i, 2) for i in range(1,5)
]
future_realtime_curve = ForecastCurve([HistogramDist(d) for d in future_realtime])

future_quantity = [
    get_slice(future_array, i, 3) for i in range(1,5)
]
future_quantity_curve = ForecastCurve([HistogramDist(d) for d in future_quantity])

futrue_submitted = [
    get_slice(future_array, i, 4) for i in range(1,5)
]

future_market = ForecastMarket(future_dayahead_curve, future_realtime_curve, future_quantity_curve)

In [31]:
print([i[1] for i in future_dayahead])

[330.25, 330.0, 330.0, 330.0]


In [37]:
from finance_utils.electrical.china.spot.rule.settlement import province_new_energy_with_recycle

print([
    province_new_energy_with_recycle(
        [x[i] for x in future_dayahead], 
        [x[i] for x in future_realtime], 
        [x[i] for x in future_quantity],
        [x[i] for x in futrue_submitted],
        trigger_rate=trigger_rate, punishment_rate=punishment_rate
    ) for i in range(len(future_dayahead[0]))
])

[915.2864987411498, 1481.587818568039, 2408.6705176902387, 1234.6870000624658, 418.67112499579787, 519.83349, 749.3262167134328, 1350.4919999999997, 4367.952007011416, 4508.827915000001, 1607.256999809265, 5620.0585, 709.9830010693073, 505.557, 364.58733505245203, 0.0, 0.0, 0.0, 0.0, 643.0296735047527, 675.640000166893, 486.31900000536444, 481.6675000327825, 722.7585435548209, 858.2445001916885]


In [38]:
print([
    province_new_energy_with_recycle(
        [x[i] for x in future_dayahead], 
        [x[i] for x in future_realtime], 
        [x[i] for x in future_quantity],
        [19.86701772,  0.1492182 ,  6.66549994,  0.52190155],
        trigger_rate=trigger_rate, punishment_rate=punishment_rate
    ) for i in range(len(future_dayahead[0]))
])

[271.75191735000027, 1480.3546306067892, 2417.8417628860643, 1279.3647977300004, 527.1286763862503, 715.689533306, 759.9197482568803, 1370.1901301000003, 4266.196278965088, 3856.0963381152997, 3058.73955955, 5586.18783762, 535.6039161943072, 486.15967956500003, 442.85792539959937, -311.34321534000003, -701.2989068165994, 52.277052049999924, -136.57333543999943, 645.5023975741832, 286.9340300168932, 269.02928392499996, 481.2155454999996, 439.28171116282044, 852.9751777699997]
